<a href="https://colab.research.google.com/github/nogast/explore_spider/blob/main/Explore_UnifiedSKG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Playground for UnfiedSKG**

## Overview

This is the demo version of UnifiedSKG for you to play with. For full function(full dataset load, full seq2seq construction, model training, full metrics evluation...), please check out our [UnfiedSKG framework](https://github.com/hkunlp/unifiedskg) and resource collection website [unifiedskg.com](https://unifiedskg.com).😃

Steps you need to play with UnifiedSKG

1.   Copy this colab to your account
2.   Choose the task
3.   Load in trained weight(finetuned/prefix-tuning, multi-task prefix-tuning...)
4.   Input your flattened seq and have fun!🤩

FAQ:


1. Available weights see [Huggingface Platform](https://huggingface.co/hkunlp).
2. Why prefix-module is big(100MB for T5-base and 300MB for T5-large)? Because we keep the re-paramized weight for potential research. See original [Prefix-tuning paper](https://arxiv.org/abs/2101.00190) for more detail information.
3. Do we need to add prefix on text(convert to SQL, give me the answer... etc.) when input it? We recommend you have this prompt words because we train our models with that although it only have littel influence on prediction(we are not so sure actually).


## Preparation

Connect to Google Drive and cd to root path

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd /content/drive/My Drive/

Mounted at /content/drive
/content/drive/My Drive


In [4]:
!git clone --recurse-submodules https://github.com/HKUNLP/UnifiedSKG.git

fatal: destination path 'UnifiedSKG' already exists and is not an empty directory.


In [5]:
%cd UnifiedSKG

/content/drive/My Drive/UnifiedSKG


In [6]:
import sys
sys.path.append('/content/drive/My Drive/UnifiedSKG')

Install dependency(partially just for demo, see full dependency in [UnfiedSKG framework dependency](https://github.com/HKUNLP/UnifiedSKG/blob/main/py3.7pytorch1.8.yaml)

In [7]:
!pip install transformers==4.9.2
!pip install nltk
!pip install datasets==1.14.0
!pip install sentencepiece
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 31.8 MB/s 
     |████████████████████████████████| 880 kB 61.7 MB/s 
     |████████████████████████████████| 3.3 MB 53.3 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895259 sha256=76c94ec60ac53cc039ca6acb4f1e4c4e0c7a980c76fb1bf3e9594d20a73946f2
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 290 kB 29.9 MB/s 
     |████████████████████████████████| 56 kB 6.2 MB/s 
     |████████████████████████████████| 115 kB 63.3 MB/s 
     |████████████████████████████████| 212 kB 75.5 MB/s 
  

Import essential packeges



In [8]:
import os
import time
import torch
import datasets
from transformers import (
    HfArgumentParser,
    set_seed,
    AutoTokenizer
)
from utils.configue import Configure
from utils.training_arguments import WrappedSeq2SeqTrainingArguments

from filelock import FileLock
import nltk
with FileLock(".lock") as lock:
    nltk.download("punkt", quiet=True)
    nltk.download("stopwords", quiet=True)

## Play

In [9]:
def play(txt, model, tokenizer):
  print("=====❓Request=====")
  print(txt)
  tokenized_txt = tokenizer([txt], max_length=1024, padding="max_length", truncation=True)
  pred = tokenizer.batch_decode(
      model.generate(
        torch.LongTensor(tokenized_txt.data['input_ids']),
        torch.LongTensor(tokenized_txt.data['attention_mask']),
        num_beams=1, 
        max_length=256
        ), 
      skip_special_tokens=True 
  ) # More details see utils/dataset.py and utils/trainer.py
  print("=====💡Answer=====")
  print(pred)

### Spider(with cell value)

Set and load the args

In [10]:
# Set args here for runnning on notebook, we make them out here to make it more illustrative.
sys.argv = ['/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py', # This is the name of your .py launcher when you run this line of code.
            # belows are the parameters we set, take spider for example
            '--cfg', 'Salesforce/T5_base_prefix_spider_with_cell_value.cfg', 
            '--output_dir', './tmp']
parser = HfArgumentParser((WrappedSeq2SeqTrainingArguments,))
training_args, = parser.parse_args_into_dataclasses()
set_seed(training_args.seed)
args = Configure.Get(training_args.cfg)

Load tokenizer and model(21->1 multitasked prefix)

In [11]:
tokenizer = AutoTokenizer.from_pretrained("hkunlp/from_all_T5_base_prefix_spider_with_cell_value2", use_fast=False)
from models.unified.prefixtuning import Model
model = Model(args)
model.load("hkunlp/from_all_T5_base_prefix_spider_with_cell_value2")

Downloading:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

prefix-tuning sequence length is 10.


Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.01G [00:00<?, ?B/s]

play

**First schema:**

In [22]:
struct_in = "| concert_singer | stadium : stadium_id , location , name , capacity , highest , lowest , average | singer : singer_id , name , country ( France ) , song_name , song_release_year , age , is_male | concert : concert_id , concert_name , theme , stadium_id , year | singer_in_concert : concert_id , singer_id"
text_in = "what is the minimum, average, and maximum age of all singers from France?"

In [23]:
play("{}; structed knowledge: {}".format(text_in, struct_in), model, tokenizer)

=====❓Request=====
what is the minimum, average, and maximum age of all singers from France?; structed knowledge: | concert_singer | stadium : stadium_id , location , name , capacity , highest , lowest , average | singer : singer_id , name , country ( France ) , song_name , song_release_year , age , is_male | concert : concert_id , concert_name , theme , stadium_id , year | singer_in_concert : concert_id , singer_id
=====💡Answer=====
['select min(age), avg(age), max(age) from singer where country = "France"']


This is a relatively simple question, and as expected it is handled well by the algorithm.

In [26]:
struct_in = "| concert_singer | stadium : stadium_id , location , name , capacity , highest , lowest , average | singer : singer_id , name , country ( France ) , song_name , song_release_year , age , is_male | concert : concert_id , concert_name , theme , stadium_id , year | singer_in_concert : concert_id , singer_id"
# struct_in = "|author : aid , homepage , name , oid | conference : cid , homepage , name | domain : did , name | domain_author : aid , did | domain_conference : cid , did | journal : homepage , jid , name | domain_journal : did , jid | keyword : keyword , kid | domain_keyword : did , kid | publication : abstract , cid , citation_num , jid , pid , reference_num , title , year | domain_publication : did , pid | organization : continent , homepage , name , oid | publication_keyword : pid , kid | writes : aid , pid | cite : cited , citing |"
text_in = "List all song names by singers above the average age."
# text_in = "what is the name of the publication with the largest number of citations?"

In [27]:
play("{}; structed knowledge: {}".format(text_in, struct_in), model, tokenizer)

=====❓Request=====
List all song names by singers above the average age.; structed knowledge: | concert_singer | stadium : stadium_id , location , name , capacity , highest , lowest , average | singer : singer_id , name , country ( France ) , song_name , song_release_year , age , is_male | concert : concert_id , concert_name , theme , stadium_id , year | singer_in_concert : concert_id , singer_id
=====💡Answer=====
['select song_name from singer where age > (select avg(age) from singer)']


This one is a bit more complicated (two SELECTs instead of one), but steal relatively simple (no JOINs) and handles well.

In [28]:
struct_in = "| concert_singer | stadium : stadium_id , location , name , capacity , highest , lowest , average | singer : singer_id , name , country ( France ) , song_name , song_release_year , age , is_male | concert : concert_id , concert_name , theme , stadium_id , year | singer_in_concert : concert_id , singer_id"
text_in = "Show the stadium name and the number of concerts in each stadium."

In [29]:
play("{}; structed knowledge: {}".format(text_in, struct_in), model, tokenizer)

=====❓Request=====
Show the stadium name and the number of concerts in each stadium.; structed knowledge: | concert_singer | stadium : stadium_id , location , name , capacity , highest , lowest , average | singer : singer_id , name , country ( France ) , song_name , song_release_year , age , is_male | concert : concert_id , concert_name , theme , stadium_id , year | singer_in_concert : concert_id , singer_id
=====💡Answer=====
['select t2.name, count(*) from concert as t1 join stadium as t2 on t1.stadium_id = t2.stadium_id group by t1.stadium_id']


In [30]:
struct_in = "| concert_singer | stadium : stadium_id , location , name , capacity , highest , lowest , average | singer : singer_id , name , country ( France ) , song_name , song_release_year , age , is_male | concert : concert_id , concert_name , theme , stadium_id , year | singer_in_concert : concert_id , singer_id"
text_in = "Show the stadium name and capacity with most number of concerts in year 2014 or after."
# expected result: SELECT T2.name ,  T2.capacity FROM concert AS T1 JOIN stadium AS T2 ON T1.stadium_id  =  T2.stadium_id WHERE T1.year  >=  2014 GROUP BY T2.stadium_id ORDER BY count(*) DESC LIMIT 1

In [31]:
play("{}; structed knowledge: {}".format(text_in, struct_in), model, tokenizer)

=====❓Request=====
Show the stadium name and capacity with most number of concerts in year 2014 or after.; structed knowledge: | concert_singer | stadium : stadium_id , location , name , capacity , highest , lowest , average | singer : singer_id , name , country ( France ) , song_name , song_release_year , age , is_male | concert : concert_id , concert_name , theme , stadium_id , year | singer_in_concert : concert_id , singer_id
=====💡Answer=====
['select t2.name, t2.capacity from concert as t1 join stadium as t2 on t1.stadium_id = t2.stadium_id where t1.year >= 2014 group by t1.stadium_id order by count(*) desc limit 1']


This is a more complicated question since it requires a JOIN in the query, but the algorithm still manages to solve it perfectly.

In [38]:
struct_in = "| concert_singer | stadium : stadium_id , location , name , capacity , highest , lowest , average | singer : singer_id , name , country ( France ) , song_name , song_release_year , age , is_male | concert : concert_id , concert_name , theme , stadium_id , year | singer_in_concert : concert_id , singer_id"
text_in = "Find the name and location of the stadiums which some concerts happened in the years of both 2014 and 2015."

In [39]:
play("{}; structed knowledge: {}".format(text_in, struct_in), model, tokenizer)

=====❓Request=====
Find the name and location of the stadiums which some concerts happened in the years of both 2014 and 2015.; structed knowledge: | concert_singer | stadium : stadium_id , location , name , capacity , highest , lowest , average | singer : singer_id , name , country ( France ) , song_name , song_release_year , age , is_male | concert : concert_id , concert_name , theme , stadium_id , year | singer_in_concert : concert_id , singer_id
=====💡Answer=====
['select t2.name, t2.location from concert as t1 join stadium as t2 on t1.stadium_id = t2.stadium_id where t1.year = 2014 intersect select t2.name, t2.location from concert as t1 join stadium as t2 on t1.stadium_id = t2.stadium_id where t1.year = 2015']


This is quit complicated, as it requires two JOINs and an INTERSECT to get to the right answer, but the algorithm still manages to do it.

Out of curiosity, I wanted to see what would happen if I only gave the question as input, with no information at all about the schema:

In [40]:
struct_in = ""
text_in = "Find the name and location of the stadiums which some concerts happened in the years of both 2014 and 2015."

In [41]:
play("{}; structed knowledge: {}".format(text_in, struct_in), model, tokenizer)

=====❓Request=====
Find the name and location of the stadiums which some concerts happened in the years of both 2014 and 2015.; structed knowledge: 
=====💡Answer=====
['select stadium, location from concerts where year = 2014 intersect select stadium, location from 2015']


In [42]:
struct_in = ""
text_in = "what is the minimum, average, and maximum age of all singers from France?"

In [43]:
play("{}; structed knowledge: {}".format(text_in, struct_in), model, tokenizer)

=====❓Request=====
what is the minimum, average, and maximum age of all singers from France?; structed knowledge: 
=====💡Answer=====
["select min(age), avg(age), max(age) from singers where country = 'France'"]


Unsurprisingly, this is more difficult. The algorithm can still deal with the simpler question, but fails on the more complicated one.

**Second schema:**

In [44]:
struct_in = "| pets_1 | Student : StuID , LName , Fname , Age , Sex , Major , Advisor , city_code | Has_Pet : StuID , PetID | Pets : PetID , PetType , pet_age , weight"
text_in = "Find the number of dog pets that are raised by female students (with sex F)"
# SELECT count(*) FROM student AS T1 JOIN has_pet AS T2 ON T1.stuid  =  T2.stuid JOIN pets AS T3 ON T2.petid  =  T3.petid WHERE T1.sex  =  'F' AND T3.pettype  =  'dog'

In [45]:
play("{}; structed knowledge: {}".format(text_in, struct_in), model, tokenizer)

=====❓Request=====
Find the number of dog pets that are raised by female students (with sex F); structed knowledge: | pets_1 | Student : StuID , LName , Fname , Age , Sex , Major , Advisor , city_code | Has_Pet : StuID , PetID | Pets : PetID , PetType , pet_age , weight
=====💡Answer=====
["select count(*) from (select petID from Has_Pet as t1 join student as t2 on t1.StuID = t2.StuID where t2.sex = 'F') and t1.pet_age = 1"]


This is the first time that I see clear mistakes here. I was curious to see if this related to the structure of the question and in particular the passive verb (raised), which intuitively seems more dificult to me. So I tried a similar question with an active verb instead:

In [46]:
struct_in = "| pets_1 | Student : StuID , LName , Fname , Age , Sex , Major , Advisor , city_code | Has_Pet : StuID , PetID | Pets : PetID , PetType , pet_age , weight"
text_in = "Find the number of female students (with sex F) who raise dogs"

In [47]:
play("{}; structed knowledge: {}".format(text_in, struct_in), model, tokenizer)

=====❓Request=====
Find the number of female students (with sex F) who raise dogs; structed knowledge: | pets_1 | Student : StuID , LName , Fname , Age , Sex , Major , Advisor , city_code | Has_Pet : StuID , PetID | Pets : PetID , PetType , pet_age , weight
=====💡Answer=====
["select count(*) from student where sex = 'F' and major = 'Al' and pet_age = 'Pet'"]


Interestingly, this still doesn't work and fields which are not related at all, such as major, somehow become a part of the query. There also seems to be a particular problem with linking the word "dog" to the petType field. To test this a bit more, I tried a simpler question with the word dog:

In [48]:
struct_in = "| pets_1 | Student : StuID , LName , Fname , Age , Sex , Major , Advisor , city_code | Has_Pet : StuID , PetID | Pets : PetID , PetType , pet_age , weight"
text_in = "Find the weight of the youngest dog."

In [49]:
play("{}; structed knowledge: {}".format(text_in, struct_in), model, tokenizer)

=====❓Request=====
Find the weight of the youngest dog.; structed knowledge: | pets_1 | Student : StuID , LName , Fname , Age , Sex , Major , Advisor , city_code | Has_Pet : StuID , PetID | Pets : PetID , PetType , pet_age , weight
=====💡Answer=====
['select weight from Pets order by pet_age limit 1']


This is better than the previous question, but it lost the word dog altogether and just queried about at all pets. So there is some link between "dog" and "pet", but maybe not between "dog" and "petType".

**Third Schema:**

In [52]:
struct_in = "| car_1 | continents : ContId , Continent | countries : CountryId , CountryName , Continent | car_makers : Id , Maker , FullName , Country | model_list : ModelId , Maker , Model | car_names : MakeId , Model , Make | cars_data : Id , MPG , Cylinders , Edispl , Horsepower , Weight , Accelerate , Year |"
text_in = "For each continent, list its id, name, and how many countries it has"

In [53]:
play("{}; structed knowledge: {}".format(text_in, struct_in), model, tokenizer)

=====❓Request=====
For each continent, list its id, name, and how many countries it has; structed knowledge: | car_1 | continents : ContId , Continent | countries : CountryId , CountryName , Continent | car_makers : Id , Maker , FullName , Country | model_list : ModelId , Maker , Model | car_names : MakeId , Model , Make | cars_data : Id , MPG , Cylinders , Edispl , Horsepower , Weight , Accelerate , Year |
=====💡Answer=====
['select t1.Continent, t1.CountyName, count(*) from continents as t1 join countries as t2 on t1.Continent = t2.Continent group by t1.Continent']


Strangely, the query tries to read the "CountyName" field from the continents table, even though it doesn't exist and the schema is part of the input, so the model should theoretically "know" that it doesn't exist.
I tried another version of the question:

In [58]:
struct_in = "| car_1 | continents : ContId , Continent | countries : CountryId , CountryName , Continent | car_makers : Id , Maker , FullName , Country | model_list : ModelId , Maker , Model | car_names : MakeId , Model , Make | cars_data : Id , MPG , Cylinders , Edispl , Horsepower , Weight , Accelerate , Year |"
text_in = "list how many countries each continent has"

In [59]:
play("{}; structed knowledge: {}".format(text_in, struct_in), model, tokenizer)

=====❓Request=====
list how many countries each continent has; structed knowledge: | car_1 | continents : ContId , Continent | countries : CountryId , CountryName , Continent | car_makers : Id , Maker , FullName , Country | model_list : ModelId , Maker , Model | car_names : MakeId , Model , Make | cars_data : Id , MPG , Cylinders , Edispl , Horsepower , Weight , Accelerate , Year |
=====💡Answer=====
['select count(*), Continent from continents group by Continent']


The mistake from the previous question is not reproduced here.
I tried another similar question:

In [60]:
struct_in = "| car_1 | continents : ContId , Continent | countries : CountryId , CountryName , Continent | car_makers : Id , Maker , FullName , Country | model_list : ModelId , Maker , Model | car_names : MakeId , Model , Make | cars_data : Id , MPG , Cylinders , Edispl , Horsepower , Weight , Accelerate , Year |"
text_in = "list the names of the countries in each continent"

In [61]:
play("{}; structed knowledge: {}".format(text_in, struct_in), model, tokenizer)

=====❓Request=====
list the names of the countries in each continent; structed knowledge: | car_1 | continents : ContId , Continent | countries : CountryId , CountryName , Continent | car_makers : Id , Maker , FullName , Country | model_list : ModelId , Maker , Model | car_names : MakeId , Model , Make | cars_data : Id , MPG , Cylinders , Edispl , Horsepower , Weight , Accelerate , Year |
=====💡Answer=====
['select t1.Continent, t2.countryName from continents as t1 join countries as t2 on t1.Continent = t2.Continent']


Again, the mistake is not reproduced. The wrong linking between countries and CountyName only exists for specific questions.